Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [80]:
NAME = "Katie Hummel"
COLLABORATORS = ""

In [93]:
%matplotlib inline
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import math
import random

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression
from sklearn import svm


plt.style.use('ggplot')

---

# Classifying Iris Data with Your Own Logistic Regression

Due:  Wednesday March 14th 2018

### 1. Starting with the Logistic Regression class we built together, adapt it so that it can predict for multiple categories.

As we discussed in class half of you will use the one versus all strategy and the other half will do the each against the other strategy.

##### One versus All

* Setosa or Not?
* Versicolor or Not?
* Virginica or Not?

##### Each against the other

* Setosa or Versicolor
* Setosa or Virginica
* Virginica or Versicolor

Then vote.



 # One Versus All Strategy
 What is the probability that y is equal to class i given x and parametrized by theta? Run all three classifiers on the input x and pick the class i that maximizes the three--whichever value of i gives the highest probability, we then predict y to be that value.

0: setosa
1: versicolor
2: virginica

In [188]:
df = pd.read_csv('/home/faculty/millbr02/pub/iris.csv', header=None)
df = df.rename(columns={0: 'sepal_length', 1:'sepal_width', 2:'petal_length', 3:'petal_width', 4:'species'}) # renaming columns
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [189]:
df['setosa'] = [1 if x == 'Iris-setosa' else 0 for x in df['species']]
df['versicolor'] = [1 if x == 'Iris-versicolor' else 0 for x in df['species']]
df['virginica'] = [1 if x == 'Iris-virginica' else 0 for x in df['species']]
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,setosa,versicolor,virginica
0,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0


In [190]:
labelencoder = LabelEncoder()
df['species'] = labelencoder.fit_transform(df['species'])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,setosa,versicolor,virginica
0,5.1,3.5,1.4,0.2,0,1,0,0
1,4.9,3.0,1.4,0.2,0,1,0,0
2,4.7,3.2,1.3,0.2,0,1,0,0
3,4.6,3.1,1.5,0.2,0,1,0,0
4,5.0,3.6,1.4,0.2,0,1,0,0


In [191]:
X = df.iloc[:, 0:3]
y = df.iloc[:, 5:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [192]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# new = ''
# for i in range(len(y_train)):
#     print(len(y_train))

In [193]:
class GDRegression:
    def __init__(self, learn_rate=0.1):
        self.lr = learn_rate # how much we want to add or subtract the weights in theta_best
        
    def predict(self, X):
        X_new_b = np.c_[np.ones((X.shape[0],1)),X]
        pp = X_new_b.dot(self.theta_best)
        return 1/(1+np.exp(-1*pp))
    
    def score(self, X, y):
        return np.mean(np.sum(np.square(self.predict(X)-y)))
    
    # X: observed values (odor, color) ,theta_best: weight for each of those columns
    def fit(self, X, y, n=1000):
        rows,cols = X.shape
        self.theta_best = np.random.rand(cols+1)
        self.scores = []
        res = self.predict(X)
        lastbase = 0
        for i in range(n):
            base = self.score(X, y)
            self.lr = self.lr / 1.005
            for i in range(len(self.theta_best)):
                savetb = self.theta_best[i]
                self.theta_best[i] = savetb + self.lr
                s1 = self.score(X, y)
                self.theta_best[i] = savetb - self.lr
                s2 = self.score(X, y)
                self.theta_best[i] = savetb
                #print(base, s1, b2)
                if s1 < base and s1 < s2:
                    self.theta_best[i] = savetb + self.lr
                elif s2 < base and s2 <= s1:
                    self.theta_best[i] = savetb - self.lr
                    base = s2
                lastbase = base
            self.scores.append(base)

In [201]:
class MultiClass:
    def __init__(self):
        self.set = GDRegression()
        self.ver = GDRegression()
        self.vir = GDRegression()
        
    def predict(self, X):
        returnDict = {'setosa': [], 'versicolor': [], 'virginica': []}
        setPred = self.set.predict(X)
        verPred = self.ver.predict(X)
        virPred = self.vir.predict(X)
        setFix = [0 if x < 0.5 else 1 for x in setPred]
        verFix = [0 if x < 0.5 else 1 for x in verPred]
        virFix = [0 if x < 0.5 else 1 for x in virPred]
        for i in range(len(setPred)):
            if setFix[i] == 1 and verFix[i] == 0 and virFix[i] == 0:
                returnDict['setosa'].append(1)
                returnDict['versicolor'].append(0)
                returnDict['virginica'].append(0)
            elif setFix[i] == 0 and verFix[i] == 1 and virFix[i] == 0:
                returnDict['setosa'].append(0)
                returnDict['versicolor'].append(1)
                returnDict['virginica'].append(0)
            elif setFix[i] == 0 and verFix[i] == 0 and virFix[i] == 1:
                returnDict['setosa'].append(0)
                returnDict['versicolor'].append(0)
                returnDict['virginica'].append(1)
            else:
                serOrig = setPred[i]
                verOrig = verPred[i]
                virOrig = virPred[i]
                if serOrig > verOrig and serOrig > virOrig:
                    returnDict['setosa'].append(1)
                    returnDict['versicolor'].append(0)
                    returnDict['virginica'].append(0)
                elif verOrig > serOrig and verOrig > virOrig:
                    returnDict['setosa'].append(0)
                    returnDict['versicolor'].append(1)
                    returnDict['virginica'].append(0)
                elif virOrig > serOrig and virOrig > verOrig:
                    returnDict['setosa'].append(0)
                    returnDict['versicolor'].append(0)
                    returnDict['virginica'].append(1)
                    
        return pd.DataFrame(returnDict)
    
    def fit(self, X, y):
        self.set.fit(X, y['setosa'])
        self.ver.fit(X, y['versicolor'])
        self.vir.fit(X, y['virginica']) 

In [202]:
n = MultiClass()
n.fit(X_train, y_train)
y_pred = n.predict(X_test)

In [203]:
print(accuracy_score(y_test, y_pred))

0.9666666666666667
